In [ ]:
#note; sheet cusbase n SMATOP10 BRR yg prev month je guna yg ad dup/ cus_dup

In [20]:
R_DATE = 20220831
P_DATE1 = 20220731
P_DATE2 = 20220630
P_DATE3 = 20220531
P_DATE4 = 20220430
P_DATE5 = 20220331

#Added 23/6/2022 for GIL From Impaired
#year = 2022        #current year
#year1 = 202204     #current month

import numpy as np
import pandas as pd
import pandasql
import sqlite3

pd.set_option("display.max_columns", None) 
pd.set_option("display.max_colwidth", 1000) #huruf dlm column
pd.set_option("display.max_rows", 100)
pd.set_option("display.precision", 2) #2 titik perpuluhan


#dalam data source ---> business financing
BB = pd.read_excel (r'T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\02_Business_Financing\01_Portfolio\\'+str(R_DATE)+'\\Master BB_'+str(R_DATE)+'.xlsx', sheet_name='Master Raw')
BB_RDMS = pd.read_excel (r'T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\12_RDMS\\'+str(R_DATE)+'\\RDMS_BB_'+str(R_DATE)+'.xlsx')
BB_NA = pd.read_excel (r'T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\02_Business_Financing\03_Newly_Approved\\'+str(R_DATE)+'\\Loan Approval_'+str(R_DATE)+'.xlsx')
BB_EWS = pd.read_excel (r'T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\02_Business_Financing\02_Asset_Quality\\'+str(R_DATE)+'\\Asset_Quality_Master List_'+str(R_DATE)+'.xlsx', sheet_name='EWS')
BB_WATCHLIST = pd.read_excel (r'T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\02_Business_Financing\02_Asset_Quality\\'+str(R_DATE)+'\\Asset_Quality_Master List_'+str(R_DATE)+'.xlsx', sheet_name='Watchlist')
BB_SMA = pd.read_excel (r'T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\02_Business_Financing\02_Asset_Quality\\'+str(R_DATE)+'\\Asset_Quality_Master List_'+str(R_DATE)+'.xlsx', sheet_name='SMA')
BB_RnR = pd.read_excel (r'T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\02_Business_Financing\02_Asset_Quality\\'+str(R_DATE)+'\\Asset_Quality_Master List_'+str(R_DATE)+'.xlsx', sheet_name='R&R')

MIB_BB_CUS_PDATE1 = pd.read_excel (r'T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\02_Business_Financing\01_Portfolio\\'+str(P_DATE1)+'\\BB2_SUMMARY_'+str(P_DATE1)+'.xlsx', sheet_name='CUSBASE2')
MIB_BB_CUS_PDATE2 = pd.read_excel (r'T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\02_Business_Financing\01_Portfolio\\'+str(P_DATE2)+'\\BB2_SUMMARY_'+str(P_DATE2)+'.xlsx', sheet_name='CUSBASE2')
MIB_BB_CUS_PDATE3 = pd.read_excel (r'T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\02_Business_Financing\01_Portfolio\\'+str(P_DATE3)+'\\BB2_SUMMARY_'+str(P_DATE3)+'.xlsx', sheet_name='CUSBASE2')
MIB_BB_CUS_PDATE4 = pd.read_excel (r'T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\02_Business_Financing\01_Portfolio\\'+str(P_DATE4)+'\\BB2_SUMMARY_'+str(P_DATE4)+'.xlsx', sheet_name='CUSBASE2')
MIB_BB_CUS_PDATE5 = pd.read_excel (r'T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\02_Business_Financing\01_Portfolio\\'+str(P_DATE5)+'\\BB2_SUMMARY_'+str(P_DATE5)+'.xlsx', sheet_name='CUSBASE2')

Total_Impaired = pd.read_excel (r"T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\20_CRP\6.0 Impaired Loan\\"+str(R_DATE)[:4]+"\\"+str(R_DATE)[:6]+"\working\(MIB) Total_impaired_"+str(R_DATE)[:6]+".xlsx")
Total_Impaired.columns = Total_Impaired.columns.str.replace(" ", "_")

BB.columns = BB.columns.str.replace(" ", "_")
BB_RDMS.columns = BB_RDMS.columns.str.replace(" ", "_")
BB_NA.columns = BB_NA.columns.str.replace(" ", "_")
BB_EWS.columns = BB_EWS.columns.str.replace(" ", "_")
BB_WATCHLIST.columns = BB_WATCHLIST.columns.str.replace(" ", "_")
BB_SMA.columns = BB_SMA.columns.str.replace(" ", "_")
BB_RnR.columns = BB_RnR.columns.str.replace(" ", "_")


In [22]:
def RISKCD(x):
    if x >= 1 and x<= 5:
        return '01.Very Low Risk'
    elif x >= 6 and x <= 10:
        return '02.Low Risk'
    elif x >= 11 and x <= 15:
        return '03.Moderate Risk'
    elif x >= 16 and x <= 21:
        return '04.High Risk'
    elif x >= 22 and x <= 23:
        return '05.Sub Standard'
    else:
        return '06.Unrated'

def FRRCD(x):
    if x == 'A+':
        return '01.Secured'
    elif x == 'A':
        return '01.Secured'
    elif x == 'B':
        return '02.Partially Secured'
    elif x == 'C':
        return '02.Partially Secured'
    elif x == 'D':
        return '02.Partially Secured'
    elif x == 'E':
        return '02.Partially Secured'
    elif x == 'F':
        return '02.Partially Secured'
    elif x == 'G':
        return '03.Clean'
    else:
        return '03.Clean'
    
BB1 = BB.iloc[np.where((BB.Facility_Level_2!='Non-Funded Tradebills')&(BB.Facility_Level_2!='FEC (10%)'))]

BB1['ISC_NOB'] = BB1['ISC_NOB'].map(str)
BB1['ISC_NOB'] = BB1['ISC_NOB'].str.strip()
BB1['ISC_NOB'] = BB1['ISC_NOB'].str.upper()
BB1.loc[BB1.ISC_NOB.isin(['01120','0113X','01140','0119XA','0119XB','01261A','01261B','01262A','01262B','01263','01269','01273','01279','01291','01299','0130X','0141X','0144X','0145X','0146XA','0146XB','0146XC','0149X','01500','016XX','0170X','0210X','0220X','0230X','0240X','031XXA','031XXB','031XXC','031XXD','032XXA','032XXB','032XXC','1120','113X','1140','119XA','119XB','1261A','1261B','1262A','1262B','1263','1269','1273','1279','1291','1299','130X','141X','144X','145X','146XA','146XB','146XC','149X','1500','16XX','170X','210X','220X','230X','240X','31XXA','31XXB','31XXC','31XXD','32XXA','32XXB','32XXC']),'ISC_PARENT1'] = "01. Agriculture, Forestry and Fishing"
BB1.loc[BB1.ISC_NOB.isin(['05XXX','0610X','0620X','0710X','07210','07291','07299','08XXX','0910XA','0910XB','0910XC','0910XD','0910XE','0910XF','0910XG','09900', '5XXX','610X','620X','710X','7210','7291','7299','8XXX','910XA','910XB','910XC','910XD','910XE','910XF','910XG','9900']),'ISC_PARENT1'] = "02. Mining and Quarrying"
BB1.loc[BB1.ISC_NOB.isin(['1010X','1020X','1030X','10401A','10401B','10402A','10402B','10499','1050X','106XX','107XX','10800','110XXA','110XXB','12000','13XXX','14XXX','151XX','1520X','16XXX','1701X','181XX','18200','19XXXA','19XXXB','20XXX','21XXX','22XXXA','22XXXB','22XXXC','22XXXD','22XXXE','22XXXF','22XXXG','23XXXA','23XXXB','23XXXC','2410XA','2410XB','24201','24202','24209A','24209B','2431XA','2431XB','24320','251XX','25200','25910','25920','25930','2599X','26300','265XX','2670X','26999','27500','27999','28170','2819XA','2819XB','2819XC','2819XD','2819XE','28220','28299A','28299B','28299C','28299D','28299E','29XXXA','29XXXB','29XXXC','301XXA','301XXB','301XXC','301XXD','30200','30999A','30999B','30999C','3100X','32500','3290X','33XXXA','33XXXB','33XXXC','33XXXD','33XXXE','33XXXF','33XXXG','33XXXH']),'ISC_PARENT1'] =  "03. Manufacturing";
BB1.loc[BB1.ISC_NOB.isin(['3510XA','3510XB','3510XC','3510XD','3510XE','3510XF','3510XG','3520XA','3520XB','3530X']),'ISC_PARENT1'] = "04. Electricity, Gas, Steam and Air Conditioning Supply"
BB1.loc[BB1.ISC_NOB.isin(['360XX','3700X','38112','38199','382XXA','382XXB','382XXC','3830X','39000']),'ISC_PARENT1']="05. Water Supply; Sewerage, Waste Management and Remediation Activities"
BB1.loc[BB1.ISC_NOB.isin(['41001A','41001B','41001C','41001D','41001E','41001F','41001G','41001H','41002A','41002B','41002C','41002D','41002E','41002F','41002G','41002H','41002I','41002J','41003','41009','4210XA','4210XB','4210XC','4210XD','4210XE','4210XF','4210XG','4220XA','4220XB','4220XC','4220XD','429XXA','429XXB','429XXC','431XX','432XX','4330X','4390X']),'ISC_PARENT1'] = "06. Construction"
BB1.loc[BB1.ISC_NOB.isin(['451XXA','451XXB','451XXC','451XXD','451XXE','4520X','4530XA','4530XB','4540X','462XXA','462XXB','462XXC','463XXA','463XXB','463XXC','463XXD','4641X','46496','46499','46510','46595','46599A','46599B','46599C','46999','4711X','4719X','472XXA','472XXB','472XXC','472XXD','478XX','479XX','40799A','40799B','40799C','40799D','40799E']),'ISC_PARENT1'] = "07. Wholesale and Retail Trade; Repair of Motor Vehicles and Motorcycles"
BB1.loc[BB1.ISC_NOB.isin(['4911X','4912X','4921X','4922X','49230','49300','5011X','50121','50122','5021X','5022X','51101A','51101B','51102','51103','51201','51202','51203','5210XA','5210XB','5221X','5222X','5223X','52241','52249','52291','52292','52299','53XXX']),'ISC_PARENT1'] = "08. Transportation and Storage"
BB1.loc[BB1.ISC_NOB.isin(['55101A','55101B','55101C','55101D','55104','55105','55108','55199','5520X','55900','56101','56102','56103','56104','56105','56106','56107','562XX','56302','56303','56399A','56399B']),'ISC_PARENT1'] = "09. Accommodation and Food Service Activities"
BB1.loc[BB1.ISC_NOB.isin(['58110','5812X','5813X','5819X','582XX','59XXX','60XXX','6110X','6120X','6130X','6190X','620XX','631XX','6391X','63990']),'ISC_PARENT1'] = "10. Information and Communication"
BB1.loc[BB1.ISC_NOB.isin(['64110','64191','64192','64193','64194','64195','64199','64200','64301','64302','64303','64304','64309','64910','64921','64922','64923','64924','64925','64929','64991','64992','64993','64999','651XX','6520X','65301','65302','66111','66112','66113','66114','66119','66121','66122','66123','66124','66125','66129','66191','66192','66199','66211','66212','66221','66222','66223','66224','66290','66301','66302','66303']),'ISC_PARENT1'] = "11. Financial and Insurance/ Takaful Activities"
BB1.loc[BB1.ISC_NOB.isin(['68101A','68101B','68101C','68102A','68102B','68102C','68102D','68102E','68103A','68103B','68104A','68104B','68104C','68104D','68104E','68104F','68104G','68109','68201','68202','68203','68209']),'ISC_PARENT1'] =  "12. Real Estate Activities"
BB1.loc[BB1.ISC_NOB.isin(['69XXX','70100','70201','70202','70203','70209','7110X','71200','72102','72103','72105','72106','72199','7220X','73100','73200','74XXX','75000']),'ISC_PARENT1'] =  "13. Professional, Scientific and Technical Activities"
BB1.loc[BB1.ISC_NOB.isin(['7710X','772XX','77301','77307','77399','77400','78XXX','79XXX','80XXX','81XXX','821XX','82200','8230X','82910','8292XA','8292XB','8292XC','82990']),'ISC_PARENT1'] = "14. Administrative and Support Service Activities"
BB1.loc[BB1.ISC_NOB.isin(['841XX','842XX','8430X']),'ISC_PARENT1'] = "15. Public Administration and Defence; Compulsory Social Security"
BB1.loc[BB1.ISC_NOB.isin(['8510X','852XX','8530X','854XX','8550X']),'ISC_PARENT1'] = "16. Education"
BB1.loc[BB1.ISC_NOB.isin(['861XX','8620X','8690X','87XXX','88XXX']),'ISC_PARENT1'] = "17. Human Health and Social Work Activities"
BB1.loc[BB1.ISC_NOB.isin(['9000X','910XX','92000','93XXX']),'ISC_PARENT1'] = "18. Arts, Entertainment and Recreation"
BB1.loc[BB1.ISC_NOB.isin(['94XXXA','94XXXB','95XXX','960XX']),'ISC_PARENT1'] = "19. Other Service Activities"
BB1.loc[BB1.ISC_NOB.isin(['TXXXX']),'ISC_PARENT1'] =  "20. Activities of Households as Employers; Undifferentiated Goods and Services Producing Activities of Households for Own Use"
BB1.loc[BB1.ISC_NOB.isin(['99000']),'ISC_PARENT1'] = "21. Activities of Extraterritorial Organisations and Bodies"
BB1.loc[BB1.ISC_NOB.isin(['97000']),'ISC_PARENT1'] = "22. Household"
BB1.ISC_PARENT1.fillna(-9999, inplace=True)
BB1.loc[BB1.ISC_NOB.isin([-9999]),'ISC_PARENT1'] = "23. Not Populated"


In [23]:
BB1.head(1)

,No,REPORT_DATE,Account_Num,acct,Account_Type_Cd,GCIF,Borrower_Name,outstanding_balance,RM'Mil,MBB_Islamic_Ind,Facility_Level_2,Funded_non_funded,ISC_NOB,ISC_DESC,ISC_PARENT,NOB,NOB_Code,Broad_Code,Broad_Sector,NOB_Desc,Subsector,BC,REGION,NOB_Parent_Cd,NPL_Date,NPL_Indicator,PL_NPL,PL_IPL_NPL,PL+GIL,BRR,FRR,Risk_Cat,Risk_Cat_CRD,Risk_Cat_GCMC,SPL_MENTION_ACCT_IND,BORW_RISK_RATG_AFTER_OVRD,BORW_RISK_RATG_BEF_OVRD,RISK_ADJ_BRR,RAW_RATG,SPRTER_ADJ_RATG,FAC_RISK_RATG,RISK_RATG_DT,Ageing,BNM_ECONMC_SECTR,WRT_OFF_GLOBL_AMT,CR_LMT_APPR_GLOBL_AMT,rwa_before,rwa,ead,el_myr,capital_requirement,MBB_Sub_Market_Segment_Desc,Customer_Class,Cust_Type,Account_Status,Account_Branch_Cd,CRD_NUM,CRD_STS,ISC_PARENT1
0,1,31/08/2022,463028180236,463028180236,H1_OTHLNS,16083088,JASA MERIN (MALAYSIA) SDN. BHD.,2.04e+08,203.97,I,Term Loan,Funded,50122,"TRANSPORT BY TOWING OR PUSHING OF BARGES, OIL RIGS",08. TRANSPORTATION AND STORAGE,NaN,7122,7000,"TRANSPORT, STORAGE AND COMMUNICATION",OF WHICH - FREIGHT TRANSPORT BY SEA-GOING & COASTAL WATER VESSELS (MRS),OF WHICH - FREIGHT TRANSPORT BY SEA-GOING & COASTAL WATER VESSELS (MRS),KEMAMAN BC,PAHANG/KELANTAN/TRENGGANU,NaN,NaN,N,PL,PL,PL,17,D,High Risk,High Risk,High Risk,NaN,NaN,17.0,NaN,19.0,17.0,D,30/03/2022,Current,7121,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PRIVATE LTD CO (LI),NaN,WLS,NaN,NaN,NaN,08. Transportation and Storage


In [166]:
#case for may2022
#use if Account_num is scientific integer
#BB1['Account_Num'] = BB1['Account_Num'].astype(str).str[:-2]

In [24]:
#--------------------------------------------------BB_ACCT------------------------------------------------

BB_ACCT1 = BB1.drop('ISC_PARENT', axis = 1)

BB_ACCT1 = BB_ACCT1.rename(columns = {'ISC_PARENT1':'ISC_PARENT'})
BB_ACCT1.loc[BB_ACCT1['BRR'] == 'Unrated', "BRR"] = 0
BB_ACCT1['BRR'] = BB_ACCT1['BRR'].map(int)

BB_ACCT1['NOA'] = 1
BB_ACCT1['OSM'] = BB_ACCT1['outstanding_balance']/1000000

Total_Impaired.V_ORIGINAL_ACCOUNT_NUMBER = Total_Impaired.V_ORIGINAL_ACCOUNT_NUMBER.astype(str)
Total_Impaired.V_ORIGINAL_ACCOUNT_NUMBER = Total_Impaired.V_ORIGINAL_ACCOUNT_NUMBER.str.upper()
Total_Impaired.V_ORIGINAL_ACCOUNT_NUMBER = Total_Impaired.V_ORIGINAL_ACCOUNT_NUMBER.str.strip()

BB_ACCT1.Account_Num = BB_ACCT1.Account_Num.astype(str)
BB_ACCT1.Account_Num = BB_ACCT1.Account_Num.str.upper()
BB_ACCT1.Account_Num = BB_ACCT1.Account_Num.str.strip()

#usual case
BB_ACCT = BB_ACCT1.merge(Total_Impaired[['V_ORIGINAL_ACCOUNT_NUMBER','GIL']].rename(columns={'V_ORIGINAL_ACCOUNT_NUMBER': 'Account_Num'}), on = 'Account_Num', how = 'left')

#casefor may2022
#BB_ACCT = BB_ACCT1.merge(Total_Impaired[['V_ORIGINAL_ACCOUNT_NUMBER','GIL']], on = 'V_ORIGINAL_ACCOUNT_NUMBER', how = 'left')

BB_ACCT.GIL.fillna(0, inplace=True)

BB_ACCT['GIF'] = BB_ACCT.GIL

BB_ACCT.drop(['GIL'], axis = 1, inplace=True)


BB_ACCT['PL_IPL_NPL'] = BB_ACCT['PL_IPL_NPL'].str.strip()
BB_ACCT['PL_IPL_NPL'] = BB_ACCT['PL_IPL_NPL'].str.upper()

BB_ACCT.loc[BB_ACCT.PL_IPL_NPL.isin(["IPL","NPL","IPL R&R"]),'NOA_GIL'] = 1
BB_ACCT.loc[~(BB_ACCT.PL_IPL_NPL.isin(["IPL","NPL","IPL R&R"])),'NOA_GIL'] = 0

BB_ACCT.loc[BB_ACCT.PL_IPL_NPL.isin(["IPL","NPL","IPL R&R"]),'OSM_GIL'] = BB_ACCT['GIF']
BB_ACCT.loc[~(BB_ACCT.PL_IPL_NPL.isin(["IPL","NPL","IPL R&R"])),'OSM_GIL'] = 0

#BB_ACCT.loc[BB_ACCT.PL_IPL_NPL.isin(["IPL","NPL","IPL R&R"]),'FINAL_RATING'] = 23
#BB_ACCT.loc[~(BB_ACCT.PL_IPL_NPL.isin(["IPL","NPL","IPL R&R"])),'FINAL_RATING'] = BB_ACCT['BRR']

BB_ACCT['FINAL_RATING'] = BB_ACCT['BRR']

BB_ACCT['FINAL_RATING'] = BB_ACCT['FINAL_RATING'].map(int)

BB_ACCT['RISK_CAT'] = BB_ACCT.FINAL_RATING.apply(RISKCD)
BB_ACCT.loc[BB_ACCT.PL_IPL_NPL.isin(["IPL","NPL","IPL R&R"]),'RISK_CAT'] = '05.Sub Standard'

BB_ACCT['COLLATERAL_TYPE'] = BB_ACCT.FRR.apply(FRRCD)

BB_ACCT.rename(columns={'Risk_Cat': 'Risk_Cat1'}, inplace=True)

#----------------------------------------------/*CUSTOMER LEVEL TABLE*/

#BB_ACCT.MBB_Islamic_Ind.fillna('NA',inplace=True)
#BB_ACCT.GCIF.fillna(0,inplace=True)
#BB_ACCT.Borrower_Name.fillna('NA',inplace=True)
#BB_ACCT.BRR.fillna(0,inplace=True)
#BB_ACCT.FRR.fillna('NA',inplace=True)
#BB_ACCT.ISC_NOB.fillna('NA',inplace=True)
#BB_ACCT.ISC_PARENT.fillna('NA',inplace=True)
#BB_ACCT.Broad_Code.fillna(0,inplace=True)
#BB_ACCT.Broad_Sector.fillna('NA',inplace=True)
#BB_ACCT.REGION.fillna('NA',inplace=True)
#BB_ACCT.BC.fillna('NA',inplace=True)
#BB_ACCT.COLLATERAL_TYPE.fillna('NA',inplace=True)
#BB_ACCT.FINAL_RATING.fillna(0,inplace=True)
#BB_ACCT.outstanding_balance.fillna(0,inplace=True)
#BB_ACCT.GIF.fillna(0,inplace=True)


row = ['MBB_Islamic_Ind','GCIF','Borrower_Name','BRR','FRR','ISC_NOB','ISC_PARENT','Broad_Code','Broad_Sector','REGION','BC','RISK_CAT','COLLATERAL_TYPE']

column = ['GIF','outstanding_balance']

C000_2 = BB_ACCT.fillna(0).groupby(row)[column].sum().reset_index().rename(columns={'Broad_Code':'SECTOR_CODE','outstanding_balance':'CUS_BAL'})

C000_1 = C000_2.iloc[np.where(C000_2.MBB_Islamic_Ind =='I')]

C000 = C000_1[['MBB_Islamic_Ind','GCIF','Borrower_Name','BRR','FRR','ISC_NOB','ISC_PARENT','SECTOR_CODE','Broad_Sector','REGION','BC','RISK_CAT','COLLATERAL_TYPE','GIF', 'CUS_BAL']]


In [25]:
#------------------------------------/*CUSTOMER LEVEL TABLE WITHOUT DUPLICATION*/


#row1 = ['MBB_Islamic_Ind','GCIF','Borrower_Name','BRR','FRR','ISC_NOB','ISC_PARENT','Broad_Code','Broad_Sector','REGION','BC','COLLATERAL_TYPE']

#column1 = ['FINAL_RATING']

#column2 = ['outstanding_balance','OSM_GIL']

#C005_1 = BB_ACCT.groupby(row1).agg({'FINAL_RATING': 'max', 'outstanding_balance': 'sum', 'OSM_GIL': 'sum'}).reset_index()

#C005_2 = C005_1.rename(columns={'Broad_Code':'SECTOR_CODE','outstanding_balance':'CUS_BAL','FINAL_RATING':'RATING','OSM_GIL':'GIL_BAL'})

#C005 = C005_2.iloc[np.where(C005_2.MBB_Islamic_Ind =='I')]


In [26]:
#------------------------------------/*CUSTOMER LEVEL TABLE WITHOUT DUPLICATION*/

row1 = ['MBB_Islamic_Ind','GCIF','Borrower_Name','BRR','FRR','ISC_NOB','ISC_PARENT','Broad_Code','Broad_Sector','REGION','BC','COLLATERAL_TYPE'] #,'FINAL_RATING'

column1 = ['FINAL_RATING']

column2 = ['outstanding_balance','OSM_GIL']

C005_1 = BB_ACCT.fillna(0).groupby(row1)[column1].max().reset_index()

C005_2 = BB_ACCT.fillna(0).groupby('GCIF')[column2].sum().reset_index()

C005_3 = C005_1.merge(C005_2,on='GCIF',how='left').drop_duplicates('GCIF').rename(columns={'Broad_Code':'SECTOR_CODE','outstanding_balance':'CUS_BAL','FINAL_RATING':'RATING','OSM_GIL':'GIL_BAL'})

C005 = C005_3.iloc[np.where(C005_3.MBB_Islamic_Ind =='I')]

#BB_ACCT.MBB_Islamic_Ind.fillna('NA',inplace=True)
#BB_ACCT.GCIF.fillna(0,inplace=True)
#BB_ACCT.Borrower_Name.fillna('NA',inplace=True)
#BB_ACCT.BRR.fillna(0,inplace=True)
#BB_ACCT.FRR.fillna('NA',inplace=True)
#BB_ACCT.ISC_NOB.fillna('NA',inplace=True)
#BB_ACCT.ISC_PARENT.fillna('NA',inplace=True)
#BB_ACCT.Broad_Code.fillna(0,inplace=True)
#BB_ACCT.Broad_Sector.fillna('NA',inplace=True)
#BB_ACCT.REGION.fillna('NA',inplace=True)
#BB_ACCT.BC.fillna('NA',inplace=True)
#BB_ACCT.COLLATERAL_TYPE.fillna('NA',inplace=True)
#BB_ACCT.FINAL_RATING.fillna(0,inplace=True)
#BB_ACCT.outstanding_balance.fillna(0,inplace=True)
#BB_ACCT.OSM_GIL.fillna(0,inplace=True)

#C005_1 = BB_ACCT.fillna(0).groupby(row1)[column2].sum().reset_index()
#C005_1 = BB_ACCT.groupby(row1).agg({'FINAL_RATING': 'max', 'outstanding_balance': 'sum', 'OSM_GIL': 'sum'}).reset_index()



In [27]:
#--------------------------------------------------BB_RDMS-------------------------------------------

BB_RDMS['GCIF'] =  BB_RDMS.COUNTERPARTY_ORIGINAL*1 

RDMS_001 = BB_RDMS.sort_values('GCIF', ascending=True).fillna(0)

RDMS_CUS_ALL1 = RDMS_001.groupby(['DEALBOOK','GCIF'])[['EAD','RWA','EL']].sum().reset_index().rename(columns={'DEALBOOK':'Organizational_Unit','EAD':'CUS_EAD','RWA':'CUS_RWA','EL':'CUS_EL'})

RDMS_CUS_ALL = RDMS_CUS_ALL1.iloc[np.where(RDMS_CUS_ALL1.Organizational_Unit.isin({"MIB_OTH","MIB_RA"}))]

C001 = C005.merge(RDMS_CUS_ALL[['GCIF','CUS_EAD','CUS_RWA','CUS_EL']],on='GCIF',how='left')

C002_1 = C001.merge(BB_ACCT[['GCIF','PL_IPL_NPL']].drop_duplicates('GCIF'),on='GCIF',how='left')

C002_1['RISK_CAT'] = C002_1.RATING.apply(RISKCD)
C002_1.loc[C002_1.PL_IPL_NPL.isin(["IPL","NPL","IPL R&R"]),'RISK_CAT'] = '05.Sub Standard'
C002 = C002_1.drop('PL_IPL_NPL', axis = 1).drop_duplicates()

C002['NOC'] = 1
C002['GIL_OSM'] = C002['GIL_BAL']/1000000
C002['CUS_OSM'] = C002['CUS_BAL']/1000000


In [28]:

#----------------------------------------/*TOP30 BY BB*/------------------------------------------------------

#----------------------------------------MIB_BB_CUS_NODUP

C002['TOP_RANK'] = C002['CUS_BAL'].rank(ascending=False)

MIB_BB_CUS_NODUP = C002.iloc[np.where(C002.TOP_RANK <= 30)]
MIB_BB_CUS_NODUP['NOC_TOP30'] = 1 
MIB_BB_CUS_NODUP['CUS_OSM_TOP30'] = MIB_BB_CUS_NODUP['CUS_BAL']/1000000 
MIB_BB_CUS_NODUP['GIL_OSM_TOP30'] = MIB_BB_CUS_NODUP['GIL_BAL']/1000000 
MIB_BB_CUS_NODUP.loc[MIB_BB_CUS_NODUP.GIL_OSM_TOP30 != 0,'NOC_GIL_TOP30'] = 1
MIB_BB_CUS_NODUP.fillna(0, inplace=True)

#----------------------------------------MIB_BB_CUS------------------------------------------------------

MIB_BB_CUS = C000
MIB_BB_CUS['NOC'] = 1
MIB_BB_CUS['CUS_OSM'] = MIB_BB_CUS['CUS_BAL']/1000000

#------------------------/*CUSTOMER LEVEL TABLE FOR IMPAIRED ACCT ONLY*/---------------------------------


list_column = ['MBB_Islamic_Ind','GCIF','Borrower_Name','BRR','FRR','ISC_NOB','ISC_PARENT','Broad_Code','Broad_Sector','REGION','BC','RISK_CAT','COLLATERAL_TYPE',]

GIL001 = BB_ACCT.iloc[np.where((BB_ACCT.MBB_Islamic_Ind == "I")&(BB_ACCT.NOA_GIL==1))]

GIL001 = GIL001.fillna(0).groupby(list_column)[['outstanding_balance','OSM_GIL']].sum().reset_index().rename(columns={'Broad_Code':'SECTOR_CODE','outstanding_balance':'CUS_BAL'})

GIL002 = GIL001

GIL002['NOC'] = 1
GIL002['CUS_OSM'] = GIL002['CUS_BAL']/1000000
GIL002['OSM_GIL'] = GIL002['OSM_GIL']/1000000

#GIL002.fillna(0, inplace=True)

GIL002.GCIF = GIL002.GCIF.astype(str)
GIL002.GCIF = GIL002.GCIF.str.upper()
GIL002.GCIF = GIL002.GCIF.str.strip()

MIB_BB_CUS_NODUP.GCIF = MIB_BB_CUS_NODUP.GCIF.astype(str)
MIB_BB_CUS_NODUP.GCIF = MIB_BB_CUS_NODUP.GCIF.str.upper()
MIB_BB_CUS_NODUP.GCIF = MIB_BB_CUS_NODUP.GCIF.str.strip()

MIB_BB_GILCUS = GIL002.merge(MIB_BB_CUS_NODUP[['GCIF','NOC_TOP30','GIL_OSM_TOP30']], on='GCIF',how='left')



D:\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
D:\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
D:\anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

In [29]:
#-------------------------------------------RISK_CAT

#&COL
RISK_CAT = C002.iloc[np.where(C002.MBB_Islamic_Ind == 'I')]
RISK_CAT = RISK_CAT.groupby(['RISK_CAT'])[['RISK_CAT','NOC','CUS_OSM']].sum().reset_index()


#GIL&COL
RISK_CAT2 = MIB_BB_GILCUS.iloc[np.where(MIB_BB_GILCUS.MBB_Islamic_Ind == 'I')]
RISK_CAT2 = RISK_CAT2.groupby(['RISK_CAT'])[['RISK_CAT','NOC','CUS_OSM','OSM_GIL','NOC_TOP30','GIL_OSM_TOP30']].sum().reset_index()

#TOP30&COL

RISK_CAT3 = C002.groupby(['RISK_CAT']).sum().reset_index()
RISK_CAT3 = RISK_CAT3[['RISK_CAT','NOC','CUS_OSM','CUS_EAD','CUS_RWA','CUS_EL']]

RISK_CAT_NODUP1 = MIB_BB_CUS_NODUP.iloc[np.where(MIB_BB_CUS_NODUP.MBB_Islamic_Ind == 'I')]
RISK_CAT_NODUP1 = RISK_CAT_NODUP1.groupby(['RISK_CAT']).sum().reset_index()
RISK_CAT_NODUP1 = RISK_CAT_NODUP1[['RISK_CAT','NOC_TOP30','CUS_OSM_TOP30','NOC_GIL_TOP30','GIL_OSM_TOP30']]
RISK_CAT4 = RISK_CAT3.merge(RISK_CAT_NODUP1, on='RISK_CAT', how='left')

summary = ['RISK_CAT','NOC','CUS_OSM','NOC_TOP30','CUS_OSM_TOP30','NOC_GIL_TOP30','GIL_OSM_TOP30','CUS_EAD','CUS_RWA','CUS_EL']
RISK_CAT4 = RISK_CAT4[summary]
RISK_CAT4 = RISK_CAT4.fillna(0)

In [30]:
#-------------------------------------------COLLATERAL_TYPE

#&COL
COLLATERAL_TYPE = C002.iloc[np.where(C002.MBB_Islamic_Ind == 'I')]
COLLATERAL_TYPE = COLLATERAL_TYPE.groupby(['COLLATERAL_TYPE']).sum().reset_index()
COLLATERAL_TYPE = COLLATERAL_TYPE[['COLLATERAL_TYPE','NOC','CUS_OSM']]

#GIL&COL
COLLATERAL_TYPE2 = MIB_BB_GILCUS.iloc[np.where(MIB_BB_GILCUS.MBB_Islamic_Ind == 'I')]
COLLATERAL_TYPE2 = COLLATERAL_TYPE2.groupby(['COLLATERAL_TYPE']).sum().reset_index()
COLLATERAL_TYPE2 = COLLATERAL_TYPE2[['COLLATERAL_TYPE','NOC','CUS_OSM','OSM_GIL','NOC_TOP30','GIL_OSM_TOP30']]

#TOP30&COL
COLLATERAL_TYPE3 = C002[['COLLATERAL_TYPE','NOC','CUS_OSM','CUS_EAD','CUS_RWA','CUS_EL']]
COLLATERAL_TYPE3 = COLLATERAL_TYPE3.groupby(['COLLATERAL_TYPE']).sum().reset_index()

COLLATERAL_TYPE_NODUP = MIB_BB_CUS_NODUP.iloc[np.where(MIB_BB_CUS_NODUP.MBB_Islamic_Ind == 'I')]
COLLATERAL_TYPE_NODUP = COLLATERAL_TYPE_NODUP.groupby(['COLLATERAL_TYPE']).sum().reset_index()
COLLATERAL_TYPE_NODUP = COLLATERAL_TYPE_NODUP[['COLLATERAL_TYPE','NOC_TOP30','CUS_OSM_TOP30','NOC_GIL_TOP30','GIL_OSM_TOP30']]
COLLATERAL_TYPE4 = COLLATERAL_TYPE3.merge(COLLATERAL_TYPE_NODUP, on='COLLATERAL_TYPE', how='left')

summary = ['COLLATERAL_TYPE','NOC','CUS_OSM','NOC_TOP30','CUS_OSM_TOP30','NOC_GIL_TOP30','GIL_OSM_TOP30','CUS_EAD','CUS_RWA','CUS_EL']
COLLATERAL_TYPE4 = COLLATERAL_TYPE4[summary]
COLLATERAL_TYPE4 = COLLATERAL_TYPE4.fillna(0)

#-------------------------------------------REGION

#&COL
REGION = C002.iloc[np.where(C002.MBB_Islamic_Ind == 'I')]
REGION = REGION.groupby(['REGION']).sum().reset_index()
REGION = REGION[['REGION','NOC','CUS_OSM']]


#GIL&COL
REGION2 = MIB_BB_GILCUS.iloc[np.where(MIB_BB_GILCUS.MBB_Islamic_Ind == 'I')]
REGION2 = REGION2.groupby(['REGION']).sum().reset_index()
REGION2 = REGION2[['REGION','NOC','CUS_OSM','OSM_GIL','NOC_TOP30','GIL_OSM_TOP30']]


#TOP30&COL
REGION3 = C002[['REGION','NOC','CUS_OSM','CUS_EAD','CUS_RWA','CUS_EL']]
REGION3 = REGION3.groupby(['REGION']).sum().reset_index()

REGION_NODUP = MIB_BB_CUS_NODUP.iloc[np.where(MIB_BB_CUS_NODUP.MBB_Islamic_Ind == 'I')]
REGION_NODUP = REGION_NODUP.groupby(['REGION']).sum().reset_index()
REGION_NODUP = REGION_NODUP[['REGION','NOC_TOP30','CUS_OSM_TOP30','NOC_GIL_TOP30','GIL_OSM_TOP30']]
REGION4 = REGION3.merge(REGION_NODUP, on='REGION', how='left')

summary = ['REGION','NOC','CUS_OSM','NOC_TOP30','CUS_OSM_TOP30','NOC_GIL_TOP30','GIL_OSM_TOP30','CUS_EAD','CUS_RWA','CUS_EL']
REGION4 = REGION4[summary]
REGION4 = REGION4.fillna(0)

#-------------------------------------------BC

#&COL
BC = C002.iloc[np.where(C002.MBB_Islamic_Ind == 'I')]
BC = BC.groupby(['BC']).sum().reset_index()
BC = BC[['BC','NOC','CUS_OSM']]

#GIL&COL
BC2 = MIB_BB_GILCUS.iloc[np.where(MIB_BB_GILCUS.MBB_Islamic_Ind == 'I')]
BC2 = BC2.groupby(['BC']).sum().reset_index()
BC2 = BC2[['BC','NOC','CUS_OSM','OSM_GIL','NOC_TOP30','GIL_OSM_TOP30']]

#TOP30&COL
BC3 = C002[['BC','NOC','CUS_OSM','CUS_EAD','CUS_RWA','CUS_EL']]
BC3 = BC3.groupby(['BC']).sum().reset_index()

BC_NODUP = MIB_BB_CUS_NODUP.iloc[np.where(MIB_BB_CUS_NODUP.MBB_Islamic_Ind == 'I')]
BC_NODUP = BC_NODUP.groupby(['BC']).sum().reset_index()
BC_NODUP = BC_NODUP[['BC','NOC_TOP30','CUS_OSM_TOP30','NOC_GIL_TOP30','GIL_OSM_TOP30']]
BC4 = BC3.merge(BC_NODUP, on='BC', how='left')

summary = ['BC','NOC','CUS_OSM','NOC_TOP30','CUS_OSM_TOP30','NOC_GIL_TOP30','GIL_OSM_TOP30','CUS_EAD','CUS_RWA','CUS_EL']
BC4 = BC4[summary]
BC4 = BC4.fillna(0)

#-------------------------------------------SECTOR_CODE

#&COL
SECTOR_CODE = C002.iloc[np.where(C002.MBB_Islamic_Ind == 'I')]
SECTOR_CODE = SECTOR_CODE.groupby(['SECTOR_CODE']).sum().reset_index()
SECTOR_CODE = SECTOR_CODE[['SECTOR_CODE','NOC','CUS_OSM']]

#GIL&COL
SECTOR_CODE2 = MIB_BB_GILCUS.iloc[np.where(MIB_BB_GILCUS.MBB_Islamic_Ind == 'I')]
SECTOR_CODE2 = SECTOR_CODE2.groupby(['SECTOR_CODE']).sum().reset_index()
SECTOR_CODE2 = SECTOR_CODE2[['SECTOR_CODE','NOC','CUS_OSM','OSM_GIL','NOC_TOP30','GIL_OSM_TOP30']]

SECTOR_CODE_NODUP = MIB_BB_CUS_NODUP.iloc[np.where(MIB_BB_CUS_NODUP.MBB_Islamic_Ind == 'I')]
SECTOR_CODE_NODUP = SECTOR_CODE_NODUP.groupby(['SECTOR_CODE']).sum().reset_index()
SECTOR_CODE_NODUP = SECTOR_CODE_NODUP[['SECTOR_CODE','NOC','CUS_OSM','NOC_TOP30','CUS_OSM_TOP30','NOC_GIL_TOP30','GIL_OSM_TOP30','CUS_EAD','CUS_RWA','CUS_EL']]

#TOP30&COL
SECTOR_CODE3 = C002[['SECTOR_CODE','NOC','CUS_OSM','CUS_EAD','CUS_RWA','CUS_EL']]
SECTOR_CODE3 = SECTOR_CODE3.groupby(['SECTOR_CODE']).sum().reset_index()

SECTOR_CODE_NODUP = MIB_BB_CUS_NODUP.iloc[np.where(MIB_BB_CUS_NODUP.MBB_Islamic_Ind == 'I')]
SECTOR_CODE_NODUP = SECTOR_CODE_NODUP.groupby(['SECTOR_CODE']).sum().reset_index()
SECTOR_CODE_NODUP = SECTOR_CODE_NODUP[['SECTOR_CODE','NOC_TOP30','CUS_OSM_TOP30','NOC_GIL_TOP30','GIL_OSM_TOP30']]
SECTOR_CODE4 = SECTOR_CODE3.merge(SECTOR_CODE_NODUP, on='SECTOR_CODE', how='left')

summary = ['SECTOR_CODE','NOC','CUS_OSM','NOC_TOP30','CUS_OSM_TOP30','NOC_GIL_TOP30','GIL_OSM_TOP30','CUS_EAD','CUS_RWA','CUS_EL']
SECTOR_CODE4 = SECTOR_CODE4[summary]
SECTOR_CODE4 = SECTOR_CODE4.fillna(0)

#-------------------------------------------ISC_PARENT

#&COL
ISC_PARENT = C002.iloc[np.where(C002.MBB_Islamic_Ind == 'I')]
ISC_PARENT = ISC_PARENT.groupby(['ISC_PARENT']).sum().reset_index()
ISC_PARENT = ISC_PARENT[['ISC_PARENT','NOC','CUS_OSM']]

#GIL&COL
ISC_PARENT2 = MIB_BB_GILCUS.iloc[np.where(MIB_BB_GILCUS.MBB_Islamic_Ind == 'I')]
ISC_PARENT2 = ISC_PARENT2.groupby(['ISC_PARENT']).sum().reset_index()
ISC_PARENT2 = ISC_PARENT2[['ISC_PARENT','NOC','CUS_OSM','OSM_GIL','NOC_TOP30','GIL_OSM_TOP30']]

#TOP30&COL
ISC_PARENT3 = C002[['ISC_PARENT','NOC','CUS_OSM','CUS_EAD','CUS_RWA','CUS_EL']]
ISC_PARENT3 = ISC_PARENT3.groupby(['ISC_PARENT']).sum().reset_index()

ISC_PARENT_NODUP = MIB_BB_CUS_NODUP.iloc[np.where(MIB_BB_CUS_NODUP.MBB_Islamic_Ind == 'I')]
ISC_PARENT_NODUP = ISC_PARENT_NODUP.groupby(['ISC_PARENT']).sum().reset_index()
ISC_PARENT_NODUP = ISC_PARENT_NODUP[['ISC_PARENT','NOC_TOP30','CUS_OSM_TOP30','NOC_GIL_TOP30','GIL_OSM_TOP30']]
ISC_PARENT4 = ISC_PARENT3.merge(ISC_PARENT_NODUP, on='ISC_PARENT', how='left')

summary = ['ISC_PARENT','NOC','CUS_OSM','NOC_TOP30','CUS_OSM_TOP30','NOC_GIL_TOP30','GIL_OSM_TOP30','CUS_EAD','CUS_RWA','CUS_EL']
ISC_PARENT4 = ISC_PARENT4[summary]
ISC_PARENT4 = ISC_PARENT4.fillna(0)


In [31]:

#--------------------------------------------------NA_MIB--------------------------------------------------

BB_NA.ENTITY = BB_NA.ENTITY.str.upper()
BB_NA.ENTITY = BB_NA.ENTITY.str.strip()
NA_MIB = BB_NA.iloc[np.where(BB_NA.ENTITY=='MIB')]
NA_MIB['NOA'] = 1
NA_MIB['RISK_CAT'] = NA_MIB.BRR.apply(RISKCD)
NA_MIB['COLLATERAL_TYPE'] = NA_MIB.FRR.apply(FRRCD)

NA_MIB['BNM_ISC_Code'] = NA_MIB['BNM_ISC_Code'].map(str)
NA_MIB['BNM_ISC_Code'] = NA_MIB['BNM_ISC_Code'].str.strip()
NA_MIB['BNM_ISC_Code'] = NA_MIB['BNM_ISC_Code'].str.upper()

NA_MIB.loc[NA_MIB.BNM_ISC_Code.isin(['01120','0113X','01140','0119XA','0119XB','01261A','01261B','01262A','01262B','01263','01269','01273','01279','01291','01299','0130X','0141X','0144X','0145X','0146XA','0146XB','0146XC','0149X','01500','016XX','0170X','0210X','0220X','0230X','0240X','031XXA','031XXB','031XXC','031XXD','032XXA','032XXB','032XXC', '1120','113X','1140','119XA','119XB','1261A','1261B','1262A','1262B','1263','1269','1273','1279','1291','1299','130X','141X','144X','145X','146XA','146XB','146XC','149X','1500','16XX','170X','210X','220X','230X','240X','31XXA','31XXB','31XXC','31XXD','32XXA','32XXB','32XXC']),'ISC_PARENT1'] = "01. Agriculture, Forestry and Fishing"
NA_MIB.loc[NA_MIB.BNM_ISC_Code.isin(['05XXX','0610X','0620X','0710X','07210','07291','07299','08XXX','0910XA','0910XB','0910XC','0910XD','0910XE','0910XF','0910XG','09900', '5XXX','610X','620X','710X','7210','7291','7299','8XXX','910XA','910XB','910XC','910XD','910XE','910XF','910XG','9900']),'ISC_PARENT1'] = "02. Mining and Quarrying"
NA_MIB.loc[NA_MIB.BNM_ISC_Code.isin(['1010X','1020X','1030X','10401A','10401B','10402A','10402B','10499','1050X','106XX','107XX','10800','110XXA','110XXB','12000','13XXX','14XXX','151XX','1520X','16XXX','1701X','181XX','18200','19XXXA','19XXXB','20XXX','21XXX','22XXXA','22XXXB','22XXXC','22XXXD','22XXXE','22XXXF','22XXXG','23XXXA','23XXXB','23XXXC','2410XA','2410XB','24201','24202','24209A','24209B','2431XA','2431XB','24320','251XX','25200','25910','25920','25930','2599X','26300','265XX','2670X','26999','27500','27999','28170','2819XA','2819XB','2819XC','2819XD','2819XE','28220','28299A','28299B','28299C','28299D','28299E','29XXXA','29XXXB','29XXXC','301XXA','301XXB','301XXC','301XXD','30200','30999A','30999B','30999C','3100X','32500','3290X','33XXXA','33XXXB','33XXXC','33XXXD','33XXXE','33XXXF','33XXXG','33XXXH']),'ISC_PARENT1'] =  "03. Manufacturing";
NA_MIB.loc[NA_MIB.BNM_ISC_Code.isin(['3510XA','3510XB','3510XC','3510XD','3510XE','3510XF','3510XG','3520XA','3520XB','3530X']),'ISC_PARENT1'] = "04. Electricity, Gas, Steam and Air Conditioning Supply"
NA_MIB.loc[NA_MIB.BNM_ISC_Code.isin(['360XX','3700X','38112','38199','382XXA','382XXB','382XXC','3830X','39000']),'ISC_PARENT1']="05. Water Supply; Sewerage, Waste Management and Remediation Activities"
NA_MIB.loc[NA_MIB.BNM_ISC_Code.isin(['41001A','41001B','41001C','41001D','41001E','41001F','41001G','41001H','41002A','41002B','41002C','41002D','41002E','41002F','41002G','41002H','41002I','41002J','41003','41009','4210XA','4210XB','4210XC','4210XD','4210XE','4210XF','4210XG','4220XA','4220XB','4220XC','4220XD','429XXA','429XXB','429XXC','431XX','432XX','4330X','4390X']),'ISC_PARENT1'] = "06. Construction"
NA_MIB.loc[NA_MIB.BNM_ISC_Code.isin(['451XXA','451XXB','451XXC','451XXD','451XXE','4520X','4530XA','4530XB','4540X','462XXA','462XXB','462XXC','463XXA','463XXB','463XXC','463XXD','4641X','46496','46499','46510','46595','46599A','46599B','46599C','46999','4711X','4719X','472XXA','472XXB','472XXC','472XXD','478XX','479XX','40799A','40799B','40799C','40799D','40799E']),'ISC_PARENT1'] = "07. Wholesale and Retail Trade; Repair of Motor Vehicles and Motorcycles"
NA_MIB.loc[NA_MIB.BNM_ISC_Code.isin(['4911X','4912X','4921X','4922X','49230','49300','5011X','50121','50122','5021X','5022X','51101A','51101B','51102','51103','51201','51202','51203','5210XA','5210XB','5221X','5222X','5223X','52241','52249','52291','52292','52299','53XXX']),'ISC_PARENT1'] = "08. Transportation and Storage"
NA_MIB.loc[NA_MIB.BNM_ISC_Code.isin(['55101A','55101B','55101C','55101D','55104','55105','55108','55199','5520X','55900','56101','56102','56103','56104','56105','56106','56107','562XX','56302','56303','56399A','56399B']),'ISC_PARENT1'] = "09. Accommodation and Food Service Activities"
NA_MIB.loc[NA_MIB.BNM_ISC_Code.isin(['58110','5812X','5813X','5819X','582XX','59XXX','60XXX','6110X','6120X','6130X','6190X','620XX','631XX','6391X','63990']),'ISC_PARENT1'] = "10. Information and Communication"
NA_MIB.loc[NA_MIB.BNM_ISC_Code.isin(['64110','64191','64192','64193','64194','64195','64199','64200','64301','64302','64303','64304','64309','64910','64921','64922','64923','64924','64925','64929','64991','64992','64993','64999','651XX','6520X','65301','65302','66111','66112','66113','66114','66119','66121','66122','66123','66124','66125','66129','66191','66192','66199','66211','66212','66221','66222','66223','66224','66290','66301','66302','66303']),'ISC_PARENT1'] = "11. Financial and Insurance/ Takaful Activities"
NA_MIB.loc[NA_MIB.BNM_ISC_Code.isin(['68101A','68101B','68101C','68102A','68102B','68102C','68102D','68102E','68103A','68103B','68104A','68104B','68104C','68104D','68104E','68104F','68104G','68109','68201','68202','68203','68209']),'ISC_PARENT1'] =  "12. Real Estate Activities"
NA_MIB.loc[NA_MIB.BNM_ISC_Code.isin(['69XXX','70100','70201','70202','70203','70209','7110X','71200','72102','72103','72105','72106','72199','7220X','73100','73200','74XXX','75000']),'ISC_PARENT1'] =  "13. Professional, Scientific and Technical Activities"
NA_MIB.loc[NA_MIB.BNM_ISC_Code.isin(['7710X','772XX','77301','77307','77399','77400','78XXX','79XXX','80XXX','81XXX','821XX','82200','8230X','82910','8292XA','8292XB','8292XC','82990']),'ISC_PARENT1'] = "14. Administrative and Support Service Activities"
NA_MIB.loc[NA_MIB.BNM_ISC_Code.isin(['841XX','842XX','8430X']),'ISC_PARENT1'] = "15. Public Administration and Defence; Compulsory Social Security"
NA_MIB.loc[NA_MIB.BNM_ISC_Code.isin(['8510X','852XX','8530X','854XX','8550X']),'ISC_PARENT1'] = "16. Education"
NA_MIB.loc[NA_MIB.BNM_ISC_Code.isin(['861XX','8620X','8690X','87XXX','88XXX']),'ISC_PARENT1'] = "17. Human Health and Social Work Activities"
NA_MIB.loc[NA_MIB.BNM_ISC_Code.isin(['9000X','910XX','92000','93XXX']),'ISC_PARENT1'] = "18. Arts, Entertainment and Recreation"
NA_MIB.loc[NA_MIB.BNM_ISC_Code.isin(['94XXXA','94XXXB','95XXX','960XX']),'ISC_PARENT1'] = "19. Other Service Activities"
NA_MIB.loc[NA_MIB.BNM_ISC_Code.isin(['TXXXX']),'ISC_PARENT1'] =  "20. Activities of Households as Employers; Undifferentiated Goods and Services Producing Activities of Households for Own Use"
NA_MIB.loc[NA_MIB.BNM_ISC_Code.isin(['99000']),'ISC_PARENT1'] = "21. Activities of Extraterritorial Organisations and Bodies"
NA_MIB.loc[NA_MIB.BNM_ISC_Code.isin(['97000']),'ISC_PARENT1'] = "22. Household"
NA_MIB.ISC_PARENT1.fillna(-9999, inplace=True)
NA_MIB.loc[NA_MIB.ISC_PARENT1.isin([-9999]),'ISC_PARENT1'] = "23. Not Populated"

NA_MIB.loc[NA_MIB['BRR'] == 'Unrated', "BRR"] = -9999
NA_MIB['BRR'] = NA_MIB['BRR'].map(int)
NA_MIB.rename(columns = {'ISC_PARENT1':'ISC_PARENT'}, inplace=True)

list_na2 = ['NAME_OF_CUSTOMER','RISK_CAT','COLLATERAL_TYPE','Region__LOS_','BC_Subspoke','BNM_ISC_Code','ISC_PARENT','FRR','BRR']
NA2_MIB = NA_MIB.groupby(list_na2)[['RM__MIL']].sum().reset_index()
NA2_MIB['NOA'] = 1

#NA&COL
NARISK_CAT = NA2_MIB.groupby(['RISK_CAT']).sum().reset_index()
NARISK_CAT = NARISK_CAT[['RISK_CAT','NOA','RM__MIL']]

NACOLLATERAL_TYPE = NA2_MIB.groupby(['COLLATERAL_TYPE']).sum().reset_index()
NACOLLATERAL_TYPE = NACOLLATERAL_TYPE[['COLLATERAL_TYPE','NOA','RM__MIL']]

NARegion__LOS_ = NA2_MIB.groupby(['Region__LOS_']).sum().reset_index()
NARegion__LOS_ = NARegion__LOS_[['Region__LOS_','NOA','RM__MIL']]

NABC_Subspoke = NA2_MIB.groupby(['BC_Subspoke']).sum().reset_index()
NABC_Subspoke = NABC_Subspoke[['BC_Subspoke','NOA','RM__MIL']]

NABNM_ISC_Code = NA2_MIB.groupby(['BNM_ISC_Code']).sum().reset_index()
NABNM_ISC_Code = NABNM_ISC_Code[['BNM_ISC_Code','NOA','RM__MIL']]

NAISC_PARENT = NA2_MIB[['ISC_PARENT','NOA','RM__MIL']]
NAISC_PARENT = NAISC_PARENT.groupby(['ISC_PARENT']).sum().reset_index()

#/*TOP10 BY NEWLY APPROVED*/
NA001 = NA_MIB[['NAME_OF_CUSTOMER','RISK_CAT','Region__LOS_','BNM_ISC_Code','ISC_PARENT','FRR','BRR','RM__MIL','NATURE_OF_BUSINESS_TEXT']].sort_values(by=['RM__MIL'],ascending=False) 
TOP10NA = NA001.head(10)

#/*TOP10 BY EWS*/
BB_EWS1 = BB_EWS.rename(columns={'GCIF_Number':'GCIF'})
BB_EWS1.Reporting_Month = BB_EWS1.Reporting_Month.astype(str)
BB_EWS1.Reporting_Month = BB_EWS1.Reporting_Month.str.replace('-', '')
BB_EWS1 = BB_EWS1.iloc[np.where(BB_EWS1.Reporting_Month == str(R_DATE))]

BB_EWS1.GCIF = BB_EWS1.GCIF.astype(str)
BB_EWS1.GCIF = BB_EWS1.GCIF.str.upper()
BB_EWS1.GCIF = BB_EWS1.GCIF.str.strip()

C002.GCIF = C002.GCIF.astype(str)
C002.GCIF = C002.GCIF.str.upper()
C002.GCIF = C002.GCIF.str.strip()

MIB_BB_EWS = BB_EWS1.merge(C002[['GCIF','RATING', 'CUS_OSM','FRR']], on='GCIF', how='inner')
EWS001 = MIB_BB_EWS[['GCIF','BORROWER','CUS_OSM','RATING','FRR']].sort_values(by=['CUS_OSM'],ascending=False)
TOP10EWS = EWS001.head(10)

#/*TOP10 BY WATCHLIST*/
BB_WL = BB_WATCHLIST.rename(columns={'GCIF_Number':'GCIF'})
BB_WL.Reporting_Month = BB_WL.Reporting_Month.astype(str)
BB_WL.Reporting_Month = BB_WL.Reporting_Month.str.replace('-', '')
BB_WL = BB_WL.iloc[np.where(BB_WL.Reporting_Month == str(R_DATE))]

BB_WL.GCIF = BB_WL.GCIF.astype(str)
BB_WL.GCIF = BB_WL.GCIF.str.upper()
BB_WL.GCIF = BB_WL.GCIF.str.strip()

MIB_BB_WL = BB_WL.merge(C002[['GCIF','RATING','CUS_OSM','FRR']], on='GCIF', how='inner')

WL001 = MIB_BB_WL[['GCIF','BORROWER','Broad_sector','Industry','REASONS_JUSTIFICATION_FOR_MITIGATION','RATING','CUS_OSM','FRR']].sort_values(by=['CUS_OSM'],ascending=False)
TOP10WATCHLIST = WL001.head(10)

#/*TOP10 BY SMA*/
BB_SMA1 = BB_SMA.rename(columns={'Acc_num':'Account_Num'})
BB_SMA1.Reporting_Month = BB_SMA1.Reporting_Month.astype(str)
BB_SMA1.Reporting_Month = BB_SMA1.Reporting_Month.str.replace('-', '')
BB_SMA1 = BB_SMA1.iloc[np.where(BB_SMA1.Reporting_Month == str(R_DATE))]

BB_SMA1.Account_Num = BB_SMA1.Account_Num.astype(str)
BB_SMA1.Account_Num = BB_SMA1.Account_Num.str.upper()
BB_SMA1.Account_Num = BB_SMA1.Account_Num.str.strip()

BB_ACCT.Account_Num = BB_ACCT.Account_Num.astype(str)
BB_ACCT.Account_Num = BB_ACCT.Account_Num.str.upper()
BB_ACCT.Account_Num = BB_ACCT.Account_Num.str.strip()

MIB_BB_SMA = BB_SMA1.merge(BB_ACCT[['Account_Num','FINAL_RATING','OSM','FRR']], on='Account_Num', how='inner')

SMA001 = MIB_BB_SMA[['GCIF_Number','BORROWER','OSM','FINAL_RATING','FRR','Broad_sector','Industry','REASON_FOR_CLASSIFCATION','ACTION_TAKEN_BY_BC']].sort_values(by=['OSM'],ascending=False)
list_sma = ['GCIF_Number','BORROWER','FINAL_RATING','FRR','Broad_sector','Industry','REASON_FOR_CLASSIFCATION','ACTION_TAKEN_BY_BC']

SMA001.fillna(0,inplace=True)
MIB_BB_SMA_CUS = SMA001.groupby(list_sma)[['OSM']].sum().reset_index()

MIB_BB_SMA_CUS = MIB_BB_SMA_CUS[['GCIF_Number','BORROWER','FINAL_RATING','FRR','Broad_sector','Industry','REASON_FOR_CLASSIFCATION','ACTION_TAKEN_BY_BC','OSM']].sort_values(by=['OSM'],ascending=False)

TOP10SMA1 = MIB_BB_SMA_CUS.head(10)

TOP10SMA12 = TOP10SMA1.merge(MIB_BB_CUS_PDATE1[['GCIF','BRR']].rename(columns={'GCIF':'GCIF_Number','BRR':'BRR_'+str(P_DATE1)}),on='GCIF_Number', how = 'left').merge(MIB_BB_CUS_PDATE2[['GCIF','BRR']].rename(columns={'GCIF':'GCIF_Number','BRR':'BRR_'+str(P_DATE2)}),on='GCIF_Number', how = 'left').merge(MIB_BB_CUS_PDATE3[['GCIF','BRR']].rename(columns={'GCIF':'GCIF_Number','BRR':'BRR_'+str(P_DATE3)}),on='GCIF_Number', how = 'left').merge(MIB_BB_CUS_PDATE4[['GCIF','BRR']].rename(columns={'GCIF':'GCIF_Number','BRR':'BRR_'+str(P_DATE4)}),on='GCIF_Number', how = 'left').merge(MIB_BB_CUS_PDATE5[['GCIF','BRR']].rename(columns={'GCIF':'GCIF_Number','BRR':'BRR_'+str(P_DATE5)}),on='GCIF_Number', how = 'left')

TOP10SMA = TOP10SMA12.fillna(0).sort_values(by=['OSM'],ascending=False) #/*just added 3rd Sep 2018*/



D:\anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [32]:
#/*RNR*/
BB_RnR1 = BB_RnR.rename(columns={'Acc_Num':'Account_Num'})
BB_RnR1.Reporting_Month = BB_RnR1.Reporting_Month.astype(str)
BB_RnR1.Reporting_Month = BB_RnR1.Reporting_Month.str.replace('-', '')
BB_RnR1 = BB_RnR1.iloc[np.where(BB_RnR1.Reporting_Month == str(R_DATE))]

BB_RnR1.Account_Num = BB_RnR1.Account_Num.astype(str)
BB_RnR1.Account_Num = BB_RnR1.Account_Num.str.upper()
BB_RnR1.Account_Num = BB_RnR1.Account_Num.str.strip()

MIB_BB_RNR = BB_RnR1.merge(BB_ACCT[['Account_Num','FINAL_RATING','OSM']], on='Account_Num', how='inner')


#/*LISTING BY RESCHEDULED*/

RESCHEDULED1 = MIB_BB_RNR.iloc[np.where(MIB_BB_RNR.AQ_Quality_Status == 'Rescheduled')]
RESCHEDULED1.columns = RESCHEDULED1.columns.str.replace("'", "_").str.replace("/", "_")
RESCHEDULED1['LIMIT'] = RESCHEDULED1['limit_(RM_000)']/1000
RESCHEDULED1['OSM'] = RESCHEDULED1['O_S(RM_000)']/1000

RESCHEDULED2 = RESCHEDULED1.fillna(0).groupby(['BORROWER','FINAL_RATING','FRR','Frequency','Remarks'])[['LIMIT','OSM']].sum().reset_index()
RESCHEDULED = RESCHEDULED2[['BORROWER','LIMIT','OSM','FINAL_RATING','FRR','Frequency','Remarks']]
RESCHEDULED = RESCHEDULED.drop_duplicates()


#/*LISTING BY RESTRUCTURED*/

RESTRUCTURED1 = MIB_BB_RNR.iloc[np.where(MIB_BB_RNR.AQ_Quality_Status == 'Restructured')]
RESTRUCTURED1.columns = RESTRUCTURED1.columns.str.replace("'", "_").str.replace("/", "_")
RESTRUCTURED1['LIMIT'] = RESTRUCTURED1['limit_(RM_000)']/1000
RESTRUCTURED1['OSM'] = RESTRUCTURED1['O_S(RM_000)']/1000

RESTRUCTURED2 = RESTRUCTURED1.fillna(0).groupby(['BORROWER','FINAL_RATING','FRR','Frequency','Remarks'])[['LIMIT','OSM']].sum().reset_index()
RESTRUCTURED = RESTRUCTURED2[['BORROWER','LIMIT','OSM','FINAL_RATING','FRR','Frequency','Remarks']]
RESTRUCTURED = RESTRUCTURED.drop_duplicates()


In [33]:
#------------------------------------------CUSBASE2-----------------------------------------

B001 = MIB_BB_CUS.sort_values('GCIF', ascending=True)

B001.GCIF = B001.GCIF.astype(str)
B001.GCIF = B001.GCIF.str.upper()
B001.GCIF = B001.GCIF.str.strip()

EWS002 = MIB_BB_EWS.sort_values('GCIF', ascending=True).drop_duplicates()
EWS002['GCIF_Number'] = EWS002.GCIF
EWS002['EWS_FLAG'] = 1

EWS002.GCIF = EWS002.GCIF.astype(str)
EWS002.GCIF = EWS002.GCIF.str.upper()
EWS002.GCIF = EWS002.GCIF.str.strip()

WL002 = MIB_BB_WL.sort_values('GCIF', ascending=True).drop_duplicates()
WL002['GCIF_Number'] = WL002.GCIF
WL002['WATCHLIST_FLAG'] = 1

WL002.GCIF = WL002.GCIF.astype(str)
WL002.GCIF = WL002.GCIF.str.upper()
WL002.GCIF = WL002.GCIF.str.strip()

SMA002 = MIB_BB_SMA_CUS.drop_duplicates().sort_values('GCIF_Number', ascending=True)
SMA002['GCIF'] = SMA002.GCIF_Number
SMA002['SMA_FLAG'] = 1

SMA002.GCIF = SMA002.GCIF.astype(str)
SMA002.GCIF = SMA002.GCIF.str.upper()
SMA002.GCIF = SMA002.GCIF.str.strip()

I002 = MIB_BB_GILCUS.sort_values('GCIF', ascending=True).drop_duplicates()
I002['IMPAIRED_FLAG'] = 1

I002.GCIF = I002.GCIF.astype(str)
I002.GCIF = I002.GCIF.str.upper()
I002.GCIF = I002.GCIF.str.strip()

RR002 = BB_RnR.sort_values('GCIF_Number', ascending=True).drop_duplicates()
RR002['GCIF'] = RR002.GCIF_Number #/*added on 8th Jan 2020*/
RR002.loc[RR002.AQ_Quality_Status == 'Rescheduled','RESCHEDULED_FLAG'] = 1
RR002.loc[RR002.AQ_Quality_Status == 'Restructured','RESTRUCTURED_FLAG'] = 1
RR002.loc[RR002.AQ_Quality_Status.isin(['Rescheduled','Restructured']),'R_N_R_FLAG'] = 1

RR002.GCIF = RR002.GCIF.astype(str)
RR002.GCIF = RR002.GCIF.str.upper()
RR002.GCIF = RR002.GCIF.str.strip()

B002 = B001.merge(RR002[['GCIF','AQ_Quality_Status']].rename(columns={'AQ_Quality_Status':'RR_TYPE'}),on='GCIF', how = 'left').merge(WL002[['GCIF','WATCHLIST_FLAG']],on='GCIF', how = 'left').merge(SMA002[['GCIF','SMA_FLAG']],on='GCIF', how = 'left').merge(I002[['GCIF','IMPAIRED_FLAG']],on='GCIF', how = 'left').merge(EWS002[['GCIF','EWS_FLAG']],on='GCIF', how = 'left').merge(RR002[['GCIF','RESCHEDULED_FLAG','RESTRUCTURED_FLAG','R_N_R_FLAG']],on='GCIF',how = 'left')
B002 = B002.fillna(0)

def CUS_STATUS(IMPAIRED_FLAG,SMA_FLAG,WATCHLIST_FLAG,EWS_FLAG):
    if IMPAIRED_FLAG == 1:
        return "IMPAIRED"
    elif SMA_FLAG == 1:
        return "SMA"
    elif WATCHLIST_FLAG == 1:
        return "WATCHLIST"
    elif EWS_FLAG == 1:
        return "EWS"
    else:
        return "PERFORMING"
B002['CUS_STATUS'] = B002.apply(lambda x: CUS_STATUS(x['IMPAIRED_FLAG'], x['SMA_FLAG'],x['WATCHLIST_FLAG'],x['EWS_FLAG']), axis=1)


def RR_STATUS(RESCHEDULED_FLAG,RESTRUCTURED_FLAG):
    if RESCHEDULED_FLAG == 1:
        return "RESCHEDULED"
    elif RESTRUCTURED_FLAG == 1:
        return "RESTRUCTURED"
    else:
        return "N"
B002['RR_STATUS'] = B002.apply(lambda x: RR_STATUS(x['RESCHEDULED_FLAG'], x['RESTRUCTURED_FLAG']), axis=1)

In [34]:
#----------------------------------------------/*MIB_BB_CUS_FINAL*/

#row = ['MBB_Islamic_Ind','GCIF','Borrower_Name','BRR','FRR','ISC_NOB','ISC_PARENT','Broad_Code','Broad_Sector','RISK_CAT','COLLATERAL_TYPE',]

#column = ['GIF','outstanding_balance']

#FIN000_1 = BB_ACCT.groupby(row)[column].sum().reset_index().rename(columns={'Broad_Code':'SECTOR_CODE','outstanding_balance':'CUS_BAL'})

#FIN000_2 = FIN000_1.merge(BB_ACCT[['Borrower_Name','REGION','BC']].drop_duplicates('Borrower_Name'), on='Borrower_Name', how='left')

#sbb ganti C002 xde dup
#FIN000 = FIN000_2.iloc[np.where(FIN000_2.MBB_Islamic_Ind =='I')].sort_values('CUS_BAL', ascending=False)#.drop_duplicates('Borrower_Name')
FIN000 = C002.iloc[np.where(C002.MBB_Islamic_Ind =='I')].sort_values('CUS_BAL', ascending=False)#.drop_duplicates('Borrower_Name')

#sbb gif ngan gil_osm
#FIN000 = FIN000.groupby(['MBB_Islamic_Ind','GCIF','Borrower_Name','BRR','FRR','ISC_NOB','ISC_PARENT','SECTOR_CODE','Broad_Sector','REGION','BC','RISK_CAT','COLLATERAL_TYPE'])[['GIF', 'CUS_BAL']].sum().reset_index()
FIN000 = FIN000.groupby(['MBB_Islamic_Ind','GCIF','Borrower_Name','BRR','FRR','ISC_NOB','ISC_PARENT','SECTOR_CODE','Broad_Sector','REGION','BC','RISK_CAT','COLLATERAL_TYPE'])[['GIL_OSM', 'CUS_BAL']].sum().reset_index()

MIB_BB_CUS_FIN = FIN000
MIB_BB_CUS_FIN['NOC'] = 1
MIB_BB_CUS_FIN['CUS_OSM'] = MIB_BB_CUS_FIN['CUS_BAL']/1000000

B001_FIN = MIB_BB_CUS_FIN.sort_values('GCIF', ascending=True)

B001_FIN.GCIF = B001_FIN.GCIF.astype(str)
B001_FIN.GCIF = B001_FIN.GCIF.str.upper()
B001_FIN.GCIF = B001_FIN.GCIF.str.strip()

B002_FIN = B001_FIN.merge(RR002[['GCIF','AQ_Quality_Status']].rename(columns={'AQ_Quality_Status':'RR_TYPE'}).drop_duplicates('GCIF'),on='GCIF', how = 'left').merge(WL002[['GCIF','WATCHLIST_FLAG']].drop_duplicates('GCIF'),on='GCIF', how = 'left').merge(SMA002[['GCIF','SMA_FLAG']].drop_duplicates('GCIF'),on='GCIF', how = 'left').merge(I002[['GCIF','IMPAIRED_FLAG']].drop_duplicates('GCIF'),on='GCIF', how = 'left').merge(EWS002[['GCIF','EWS_FLAG']].drop_duplicates('GCIF'),on='GCIF', how = 'left').merge(RR002[['GCIF','RESCHEDULED_FLAG','RESTRUCTURED_FLAG','R_N_R_FLAG']].drop_duplicates('GCIF'),on='GCIF',how = 'left')
B002_FIN = B002_FIN.fillna(0)
B002_FIN = B002_FIN.drop_duplicates()

def CUS_STATUS(IMPAIRED_FLAG,SMA_FLAG,WATCHLIST_FLAG,EWS_FLAG):
    if IMPAIRED_FLAG == 1:
        return "IMPAIRED"
    elif SMA_FLAG == 1:
        return "SMA"
    elif WATCHLIST_FLAG == 1:
        return "WATCHLIST"
    elif EWS_FLAG == 1:
        return "EWS"
    else:
        return "PERFORMING"
B002_FIN['CUS_STATUS'] = B002_FIN.apply(lambda x: CUS_STATUS(x['IMPAIRED_FLAG'], x['SMA_FLAG'],x['WATCHLIST_FLAG'],x['EWS_FLAG']), axis=1)


def RR_STATUS(RESCHEDULED_FLAG,RESTRUCTURED_FLAG):
    if RESCHEDULED_FLAG == 1:
        return "RESCHEDULED"
    elif RESTRUCTURED_FLAG == 1:
        return "RESTRUCTURED"
    else:
        return "N"
B002_FIN['RR_STATUS'] = B002_FIN.apply(lambda x: RR_STATUS(x['RESCHEDULED_FLAG'], x['RESTRUCTURED_FLAG']), axis=1)


#ISC_NOB,FRR,RISK_CAT

B002_FIN.sort_values('CUS_BAL', ascending=False, inplace=True)

B003_FIN = B002_FIN.groupby(['GCIF'])[['GIL_OSM','CUS_BAL','CUS_OSM']].sum().reset_index()
#B003_FIN = B002_FIN.groupby(['GCIF'])[['GIF','CUS_BAL','CUS_OSM']].sum().reset_index()
#sbb gif n gil_osm

B003_FINa = B002_FIN[['MBB_Islamic_Ind','GCIF','Borrower_Name','BRR','FRR','ISC_NOB','ISC_PARENT','SECTOR_CODE','Broad_Sector','REGION','BC','RISK_CAT','COLLATERAL_TYPE','NOC','RR_TYPE','WATCHLIST_FLAG','SMA_FLAG','IMPAIRED_FLAG','EWS_FLAG','RESCHEDULED_FLAG','RESTRUCTURED_FLAG','R_N_R_FLAG','CUS_STATUS','RR_STATUS']]

MIB_BB_CUS_FINAL = B003_FIN.merge(B003_FINa.drop_duplicates('GCIF', keep='first'), on='GCIF',how='left')

MIB_BB_CUS_FINAL = MIB_BB_CUS_FINAL[['MBB_Islamic_Ind','GCIF','Borrower_Name','BRR','FRR','ISC_NOB','ISC_PARENT','SECTOR_CODE','Broad_Sector','REGION','BC','RISK_CAT','COLLATERAL_TYPE','GIL_OSM','CUS_BAL','NOC','CUS_OSM','RR_TYPE','WATCHLIST_FLAG','SMA_FLAG','IMPAIRED_FLAG','EWS_FLAG','RESCHEDULED_FLAG','RESTRUCTURED_FLAG','R_N_R_FLAG','CUS_STATUS','RR_STATUS']].sort_values('GCIF', ascending=True)#.drop_duplicates('GCIF', keep='last')
#MIB_BB_CUS_FINAL = MIB_BB_CUS_FINAL[['MBB_Islamic_Ind','GCIF','Borrower_Name','BRR','FRR','ISC_NOB','ISC_PARENT','SECTOR_CODE','Broad_Sector','REGION','BC','RISK_CAT','COLLATERAL_TYPE','GIF','CUS_BAL','NOC','CUS_OSM','RR_TYPE','WATCHLIST_FLAG','SMA_FLAG','IMPAIRED_FLAG','EWS_FLAG','RESCHEDULED_FLAG','RESTRUCTURED_FLAG','R_N_R_FLAG','CUS_STATUS','RR_STATUS']].sort_values('GCIF', ascending=True)#.drop_duplicates('GCIF', keep='last')


In [37]:
#---------------------------------------------EXCEL-------------------------------------

writer = pd.ExcelWriter(r'T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\02_Business_Financing\01_Portfolio\\'+str(R_DATE)+'\\BB2_SUMMARY_'+str(R_DATE)+'(python).xlsx',engine='xlsxwriter')

RISK_CAT.to_excel(writer, sheet_name='RISK_CAT', index = False)
RISK_CAT2.to_excel(writer, sheet_name='GILRISK_CAT', index = False)
RISK_CAT4.to_excel(writer,sheet_name='TOP30RISK_CAT', index = False)

COLLATERAL_TYPE.to_excel(writer, sheet_name='COLLATERAL_TYPE', index = False)
COLLATERAL_TYPE2.to_excel(writer, sheet_name='GILCOLLATERAL_TYPE', index = False)
COLLATERAL_TYPE4.to_excel(writer,sheet_name='TOP30COLLATERAL_TYPE', index = False)

REGION.to_excel(writer, sheet_name='REGION', index = False)
REGION2.to_excel(writer, sheet_name='GILREGION', index = False)
REGION4.to_excel(writer,sheet_name='TOP30REGION', index = False)

BC.to_excel(writer, sheet_name='BC', index = False)
BC2.to_excel(writer, sheet_name='GILBC', index = False)
BC4.to_excel(writer,sheet_name='TOP30BC', index = False)

SECTOR_CODE.to_excel(writer, sheet_name='SECTOR_CODE', index = False)
SECTOR_CODE2.to_excel(writer, sheet_name='GILSECTOR_CODE', index = False)
SECTOR_CODE4.to_excel(writer,sheet_name='TOP30SECTOR_CODE', index = False)

ISC_PARENT.to_excel(writer, sheet_name='ISC_PARENT', index = False)
ISC_PARENT2.to_excel(writer, sheet_name='GILISC_PARENT', index = False)
ISC_PARENT4.to_excel(writer,sheet_name='TOP30ISC_PARENT', index = False)

NARISK_CAT.to_excel(writer,sheet_name='NARISK_CAT', index = False)
NACOLLATERAL_TYPE.to_excel(writer,sheet_name='NACOLLATERAL_TYPE', index = False)
NARegion__LOS_.to_excel(writer,sheet_name='NARegion__LOS_', index = False)
NABC_Subspoke.to_excel(writer,sheet_name='NABC_Subspoke', index = False)
NABNM_ISC_Code.to_excel(writer,sheet_name='NABNM_ISC_Code', index = False)
NAISC_PARENT.to_excel(writer,sheet_name='NAISC_PARENT', index = False)

MIB_BB_CUS_FINAL.to_excel(writer,sheet_name='CUSBASE2',index=False)
MIB_BB_CUS.to_excel(writer,sheet_name='CUSBASE',index=False)
MIB_BB_GILCUS.to_excel(writer,sheet_name='GILBASE',index=False)

TOP10NA.to_excel(writer,sheet_name='NATOP10',index=False)
MIB_BB_SMA.to_excel(writer,sheet_name='SMABASE',index=False)
MIB_BB_WL.to_excel(writer,sheet_name='WLBASE',index=False)
MIB_BB_EWS.to_excel(writer,sheet_name='EWSBASE',index=False)
TOP10EWS.to_excel(writer,sheet_name='EWSTOP10',index=False)
TOP10WATCHLIST.to_excel(writer,sheet_name='WLTOP10',index=False)
TOP10SMA.to_excel(writer,sheet_name='SMATOP10',index=False)
RESCHEDULED.to_excel(writer,sheet_name='RESCHEDULED',index=False)
RESTRUCTURED.to_excel(writer,sheet_name='RESTRUCTURED',index=False)


writer.save()

In [38]:
#sss\

In [180]:
df_RISK_CAT=pd.DataFrame({'RISK_CAT':['01.Very Low Risk',\
                                      '02.Low Risk',\
                                      '03.Moderate Risk',\
                                      '04.High Risk',\
                                      '05.Sub Standard',\
                                      '06.Unrated']})

In [181]:
RISK1 = df_RISK_CAT.merge(RISK_CAT,on='RISK_CAT',how='left').fillna(0)
RISK2 = df_RISK_CAT.merge(RISK_CAT2,on='RISK_CAT',how='left').fillna(0)
RISK3 = df_RISK_CAT.merge(RISK_CAT4,on='RISK_CAT',how='left').fillna(0)


In [182]:
sum(RISK_CAT4.NOC), sum(RISK3.NOC)

(3681, 3681)

In [183]:
df_COLLATERAL_TYPE=pd.DataFrame({'COLLATERAL_TYPE':['01.Secured',\
                                                    '02.Partially Secured',\
                                                    '03.Clean']})

In [184]:
COLLATERAL1 = df_COLLATERAL_TYPE.merge(COLLATERAL_TYPE,on='COLLATERAL_TYPE',how='left').fillna(0)
COLLATERAL2 = df_COLLATERAL_TYPE.merge(COLLATERAL_TYPE2,on='COLLATERAL_TYPE',how='left').fillna(0)
COLLATERAL3 = df_COLLATERAL_TYPE.merge(COLLATERAL_TYPE4,on='COLLATERAL_TYPE',how='left').fillna(0)


In [185]:
sum(COLLATERAL_TYPE4.NOC), sum(COLLATERAL3.NOC)

(3681, 3681)

In [186]:
df_REGION=pd.DataFrame({'REGION':["FEDERAL TERRITORY",\
                                  "JOHOR/MALACCA",\
                                  "PAHANG/KELANTAN/TRENGGANU",\
                                  "PENANG/KEDAH/PERLIS",\
                                  "PERAK",\
                                  "SABAH",\
                                  "SARAWAK",\
                                  "SELANGOR/NEGERI SEMBILAN"]})

In [187]:
REG1 = df_REGION.merge(REGION,on='REGION',how='left').fillna(0)
REG2 = df_REGION.merge(REGION2,on='REGION',how='left').fillna(0)
REG3 = df_REGION.merge(REGION4,on='REGION',how='left').fillna(0)


In [188]:
sum(REGION4.NOC), sum(REG3.NOC)

(3681, 3681)

In [189]:
df_BC=pd.DataFrame({'BC':[    'ALOR SETAR BC',\
    'BANGSAR BC',\
    'BATU PAHAT BC',\
    'BINTULU BC',\
    'IPOH BC',\
    'JLN TUN PERAK BC',\
    'JLN P RAMLEE BC',\
    'JOHOR BARU BC',\
    'KAJANG BC',\
    'KARAMUNSING BC',\
    'KEMAMAN BC',\
    'KLANG BC',\
    'KOTA BHARU BC',\
    'KUALA TERENGGANU BC',\
    'KUANTAN BC',\
    'KUCHING BC',\
    'MALACCA BC',\
    'MENTAKAB BC',\
    'MIRI BC',\
    'MUAR BC',\
    'PENANG BC',\
    'PETALING JAYA BC',\
    'PRAI BC',\
    'SANDAKAN BC',\
    'SENTUL RAYA BC',\
    'SEREMBAN BC',\
    'SHAH ALAM BC',\
    'SIBU BC',\
    'SRI DAMANSARA BC',\
    'SUBANG BC',\
    'SUNGAI PETANI BC',\
    'TAWAU BC',\
    'TELUK INTAN BC'\
]})

#CASE Apr 2022 ad JOHOR BARU
#CASE May 2022 ad JOHOR BAHRU BC n JOHOR BARU BC
#CASE Jun 2022 ad JOHOR BAHRU BC

In [190]:
BC_1 = df_BC.merge(BC,on='BC',how='outer').fillna(0)
BC_2 = df_BC.merge(BC2,on='BC',how='outer').fillna(0)
BC_3 = df_BC.merge(BC4,on='BC',how='outer').fillna(0)


In [203]:
BC_3

,BC,NOC,CUS_OSM,NOC_TOP30,CUS_OSM_TOP30,NOC_GIL_TOP30,GIL_OSM_TOP30,CUS_EAD,CUS_RWA,CUS_EL
0,ALOR SETAR BC,133.0,419.06,2.0,127.60,0.0,0.00,4.83e+08,3.93e+08,7.58e+06
1,BANGSAR BC,262.0,1202.92,3.0,201.60,0.0,0.00,1.32e+09,9.92e+08,1.99e+07
2,BATU PAHAT BC,130.0,348.36,0.0,0.00,0.0,0.00,4.27e+08,2.84e+08,3.67e+06
3,BINTULU BC,51.0,87.18,0.0,0.00,0.0,0.00,1.03e+08,6.58e+07,1.43e+06
4,IPOH BC,239.0,621.68,1.0,60.56,0.0,0.00,7.01e+08,5.50e+08,7.08e+06
5,JLN TUN PERAK BC,145.0,718.30,2.0,208.16,0.0,0.00,8.07e+08,6.70e+08,8.09e+06
6,JLN P RAMLEE BC,124.0,567.35,3.0,174.00,0.0,0.00,7.36e+08,5.64e+08,4.84e+06
7,JOHOR BARU BC,0.0,0.00,0.0,0.00,0.0,0.00,0.00e+00,0.00e+00,0.00e+00
8,KAJANG BC,108.0,449.43,0.0,0.00,0.0,0.00,4.89e+08,3.57e+08,7.54e+06
9,KARAMUNSING BC,79.0,265.48,1.0,60.90,0.0,0.00,3.09e+08,2.19e+08,1.83e+07


In [191]:
sum(BC4.NOC), sum(BC_3.NOC)

(3681, 3681.0)

In [192]:
df_SECTOR_CODE=pd.DataFrame({'SECTOR_CODE':[     '1000',\
     '2000',\
     '3000',\
     '4000',\
     '5000',\
     '6000',\
     '7000',\
     '8000',\
     '9000',\
     '9999'\
]})

In [193]:
df_SECTOR_CODE.SECTOR_CODE = df_SECTOR_CODE.SECTOR_CODE.astype(str)
SECTOR_CODE.SECTOR_CODE = SECTOR_CODE.SECTOR_CODE.astype(str)
SECTOR_CODE2.SECTOR_CODE = SECTOR_CODE2.SECTOR_CODE.astype(str)
SECTOR_CODE4.SECTOR_CODE = SECTOR_CODE4.SECTOR_CODE.astype(str)

SECTOR1 = df_SECTOR_CODE.merge(SECTOR_CODE,on='SECTOR_CODE',how='left').fillna(0)
SECTOR2 = df_SECTOR_CODE.merge(SECTOR_CODE2,on='SECTOR_CODE',how='left').fillna(0)
SECTOR3 = df_SECTOR_CODE.merge(SECTOR_CODE4,on='SECTOR_CODE',how='left').fillna(0)


In [194]:
sum(SECTOR_CODE4.NOC), sum(SECTOR3.NOC)

(3681, 3681.0)

In [195]:
df_ISC_PARENT=pd.DataFrame({'ISC_PARENT':['01. Agriculture, Forestry and Fishing',\
'02. Mining and Quarrying',\
'03. Manufacturing',\
'04. Electricity, Gas, Steam and Air Conditioning Supply',\
'05. Water Supply; Sewerage, Waste Management and Remediation Activities',\
'06. Construction',\
'07. Wholesale and Retail Trade; Repair of Motor Vehicles and Motorcycles',\
'08. Transportation and Storage',\
'09. Accommodation and Food Service Activities',\
'10. Information and Communication',\
'11. Financial and Insurance/ Takaful Activities',\
'12. Real Estate Activities',\
'13. Professional, Scientific and Technical Activities',\
'14. Administrative and Support Service Activities',\
'15. Public Administration and Defence; Compulsory Social Security',\
'16. Education',\
'17. Human Health and Social Work Activities',\
'18. Arts, Entertainment and Recreation',\
'19. Other Service Activities',\
'20. Activities of Households as Employers; Undifferentiated Goods and Services Producing Activities of Households for Own Use',\
'21. Activities of Extraterritorial Organisations and Bodies',\
'22. Household',\
'23. Not Populated'\
]})

In [196]:
ISC1 = df_ISC_PARENT.merge(ISC_PARENT,on='ISC_PARENT',how='left').fillna(0)
ISC2 = df_ISC_PARENT.merge(ISC_PARENT2,on='ISC_PARENT',how='left').fillna(0)
ISC3 = df_ISC_PARENT.merge(ISC_PARENT4,on='ISC_PARENT',how='left').fillna(0)


In [197]:
sum(ISC_PARENT4.NOC), sum(ISC3.NOC)

(3681, 3681.0)

In [198]:
df_NAREGION=pd.DataFrame({'Region__LOS_':["FEDERAL TERRITORY",\
                                  "JOHOR / MELAKA",\
                                  "PAHANG / KELANTAN / TERENGGANU",\
                                  "PENANG / KEDAH / PERLIS",\
                                  "PERAK",\
                                  "SABAH",\
                                  "SARAWAK",\
                                  "SELANGOR / N.SEMBILAN",\
                                   "BD HO Team"]})

In [199]:
df_NABC=pd.DataFrame({'BC_Subspoke':['ALOR SETAR BC',\
    'BANDAR SRI DAMANSARA BC',\
    'BANGSAR BC',\
    'BATU PAHAT BC/KLUANG BC',\
    'BD HO Team',\
    'BINTULU BC',\
    'IPOH BC/TAIPING BC',\
    'JLN P RAMLEE BC/PUDU BC',\
    'JLN TUN PERAK BC',\
    'JOHOR BHARU BC/PERMAS JAYA',\
    'KAJANG BC',\
    'KARAMUNSING BC',\
    'KEMAMAN BC',\
    'KLANG BC',\
    'KOTA BAHRU BC',\
    'KUALA TERENGGANU BC',\
    'KUANTAN BC',\
    'KUCHING BC',\
    'MELAKA BC',\
    'MENTAKAB BC',\
    'MIRI BC',\
    'MUAR BC',\
    'PENANG BC',\
    'PETALING JAYA BC',\
    'PRAI BC',\
    'SANDAKAN BC',\
    'SENTUL RAYA BC',\
    'SEREMBAN BC',\
    'SHAH ALAM BC',\
    'SIBU BC',\
    'SME PLUS',\
    'SUBANG BC',\
    'SUNGAI PETANI BC',\
    'TAWAU BC',\
    'TELUK INTAN BC/SITIAWAN BC']})


In [200]:
NARISK_CAT1 = df_RISK_CAT.merge(NARISK_CAT,on='RISK_CAT',how='left').fillna(0)

NACOLLATERAL_TYPE1 = df_COLLATERAL_TYPE.merge(NACOLLATERAL_TYPE,on='COLLATERAL_TYPE',how='left').fillna(0)

NARegion__LOS_1 = df_NAREGION.merge(NARegion__LOS_,on='Region__LOS_',how='left').fillna(0)

NABC_Subspoke1 = df_NABC.merge(NABC_Subspoke,on='BC_Subspoke',how='left').fillna(0)

NAISC_PARENT1 = df_ISC_PARENT.merge(NAISC_PARENT,on='ISC_PARENT',how='left').fillna(0)


In [201]:
sum(NARISK_CAT.NOA),sum(NARISK_CAT1.NOA),\
sum(NACOLLATERAL_TYPE.NOA),sum(NACOLLATERAL_TYPE1.NOA),\
sum(NARegion__LOS_.NOA),sum(NARegion__LOS_1.NOA),\
sum(NABC_Subspoke.NOA),sum(NABC_Subspoke1.NOA),\
sum(NAISC_PARENT.NOA),sum(NAISC_PARENT1.NOA),\


(189, 189.0, 189, 189, 189, 189.0, 189, 189.0, 189, 189.0)

In [202]:
MIB_BB_GILCUS_TOP10 = MIB_BB_GILCUS.sort_values(by='CUS_OSM',ascending=False).head(10)

In [204]:
#---------------------------------------------EXCEL-------------------------------------

writer = pd.ExcelWriter(r'T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\02_Business_Financing\01_Portfolio\\'+str(R_DATE)+'\\BB2_SUMMARY_'+str(R_DATE)+'(python).xlsx',engine='xlsxwriter')

RISK1.to_excel(writer, sheet_name='RISK_CAT', index = False)
RISK2.to_excel(writer, sheet_name='GILRISK_CAT', index = False)
RISK3.to_excel(writer,sheet_name='TOP30RISK_CAT', index = False)

COLLATERAL1.to_excel(writer, sheet_name='COLLATERAL_TYPE', index = False)
COLLATERAL2.to_excel(writer, sheet_name='GILCOLLATERAL_TYPE', index = False)
COLLATERAL3.to_excel(writer,sheet_name='TOP30COLLATERAL_TYPE', index = False)

REG1.to_excel(writer, sheet_name='REGION', index = False)
REG2.to_excel(writer, sheet_name='GILREGION', index = False)
REG3.to_excel(writer,sheet_name='TOP30REGION', index = False)

BC_1.to_excel(writer, sheet_name='BC', index = False)
BC_2.to_excel(writer, sheet_name='GILBC', index = False)
BC_3.to_excel(writer,sheet_name='TOP30BC', index = False)

SECTOR1.to_excel(writer, sheet_name='SECTOR_CODE', index = False)
SECTOR2.to_excel(writer, sheet_name='GILSECTOR_CODE', index = False)
SECTOR3.to_excel(writer,sheet_name='TOP30SECTOR_CODE', index = False)

ISC1.to_excel(writer, sheet_name='ISC_PARENT', index = False)
ISC2.to_excel(writer, sheet_name='GILISC_PARENT', index = False)
ISC3.to_excel(writer,sheet_name='TOP30ISC_PARENT', index = False)

NARISK_CAT1.to_excel(writer,sheet_name='NARISK_CAT', index = False)
NACOLLATERAL_TYPE1.to_excel(writer,sheet_name='NACOLLATERAL_TYPE', index = False)
NARegion__LOS_1.to_excel(writer,sheet_name='NARegion__LOS_', index = False)
NABC_Subspoke1.to_excel(writer,sheet_name='NABC_Subspoke', index = False)
NABNM_ISC_Code.to_excel(writer,sheet_name='NABNM_ISC_Code', index = False)
NAISC_PARENT1.to_excel(writer,sheet_name='NAISC_PARENT', index = False)

MIB_BB_CUS_FINAL.to_excel(writer,sheet_name='CUSBASE2',index=False)
MIB_BB_CUS.to_excel(writer,sheet_name='CUSBASE',index=False)
MIB_BB_GILCUS.to_excel(writer,sheet_name='GILBASE',index=False)
MIB_BB_GILCUS_TOP10.to_excel(writer,sheet_name='IMPAIREDTOP10',index=False)

TOP10NA.to_excel(writer,sheet_name='NATOP10',index=False)
MIB_BB_SMA.to_excel(writer,sheet_name='SMABASE',index=False)
MIB_BB_WL.to_excel(writer,sheet_name='WLBASE',index=False)
MIB_BB_EWS.to_excel(writer,sheet_name='EWSBASE',index=False)
TOP10EWS.to_excel(writer,sheet_name='EWSTOP10',index=False)
TOP10WATCHLIST.to_excel(writer,sheet_name='WLTOP10',index=False)
TOP10SMA.to_excel(writer,sheet_name='SMATOP10',index=False)
RESCHEDULED.to_excel(writer,sheet_name='RESCHEDULED',index=False)
RESTRUCTURED.to_excel(writer,sheet_name='RESTRUCTURED',index=False)


writer.save()